In [28]:
def train_pytorch():
    import os

    import torch
    from torch import nn
    import torch.nn.functional as F

    from torchvision import datasets, transforms
    import torch.distributed as dist
    from torch.utils.data import DataLoader, DistributedSampler

    # [1] Configure CPU/GPU device and distributed backend.
    # Kubeflow Trainer will automatically configure the distributed environment.
    device, backend = ("cuda", "nccl") if torch.cuda.is_available() else ("cpu", "gloo")
    dist.init_process_group(backend=backend)

    local_rank = int(os.getenv("LOCAL_RANK", 0))
    print(
        "Distributed Training with WORLD_SIZE: {}, RANK: {}, LOCAL_RANK: {}.".format(
            dist.get_world_size(),
            dist.get_rank(),
            local_rank,
        )
    )

    # [2] Define PyTorch CNN Model to be trained.
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(1, 20, 5, 1)
            self.conv2 = nn.Conv2d(20, 50, 5, 1)
            self.fc1 = nn.Linear(4 * 4 * 50, 500)
            self.fc2 = nn.Linear(500, 10)

        def forward(self, x):
            x = F.relu(self.conv1(x))
            x = F.max_pool2d(x, 2, 2)
            x = F.relu(self.conv2(x))
            x = F.max_pool2d(x, 2, 2)
            x = x.view(-1, 4 * 4 * 50)
            x = F.relu(self.fc1(x))
            x = self.fc2(x)
            return F.log_softmax(x, dim=1)

    # [3] Attach model to the correct device.
    device = torch.device(f"{device}:{local_rank}")
    model = nn.parallel.DistributedDataParallel(Net().to(device))
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

    # [4] Get the Fashion-MNIST dataset and distributed it across all available devices.
    dataset = datasets.FashionMNIST(
        "./data",
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor()]),
    )
    train_loader = DataLoader(
        dataset,
        batch_size=100,
        sampler=DistributedSampler(dataset),
    )

    # [5] Define the training loop.
    for epoch in range(1):
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            # Attach tensors to the device.
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = F.nll_loss(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if batch_idx % 10 == 0 and dist.get_rank() == 0:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(inputs),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )

    # Wait for the training to complete and destroy to PyTorch distributed process group.
    dist.barrier()
    if dist.get_rank() == 0:
        print("Training is finished")
    dist.destroy_process_group()

In [29]:
from kubeflow.trainer import TrainerClient, CustomTrainer

for r in TrainerClient().list_runtimes():
    print(f"Runtime: {r.name}")

Runtime mpi-distributed must have trainer.kubeflow.org/framework label.
Runtime torchtune-llama3.2-1b must have trainer.kubeflow.org/framework label.
Runtime torchtune-llama3.2-3b must have trainer.kubeflow.org/framework label.


Runtime: deepspeed-distributed
Runtime: mlx-distributed
Runtime: torch-distributed
Runtime: torchtune-qwen2.5-1.5b


In [33]:
job_id = TrainerClient().train(
    trainer=CustomTrainer(
        func=train_pytorch,
        num_nodes=1,
        resources_per_node={
            "cpu": 1,
            "memory": "4Gi",
            "gpu": 1, # Comment this line if you don't have GPUs.
        },
    ),
    runtime=TrainerClient().get_runtime("torch-distributed"),
)

In [31]:
TrainerClient().list_jobs()

[TrainJob(name='s5ae0235f02c', creation_timestamp=datetime.datetime(2025, 11, 7, 18, 42, 36, tzinfo=TzInfo(0)), runtime=Runtime(name='torch-distributed', trainer=RuntimeTrainer(trainer_type=<TrainerType.CUSTOM_TRAINER: 'CustomTrainer'>, framework='torch', num_nodes=1, device='Unknown', device_count='Unknown'), pretrained_model=None), steps=[Step(name='node-0', status='Running', pod_name='s5ae0235f02c-node-0-0-sl6lp', device='gpu', device_count='1')], num_nodes=1, status='Running')]

In [32]:
for s in TrainerClient().get_job(name=job_id).steps:
    print(f"Step: {s.name}, Status: {s.status}, Devices: {s.device} x {s.device_count}")

Step: node-0, Status: Running, Devices: gpu x 1


In [25]:
for logline in TrainerClient().get_job_logs(job_id, follow=True):
    print(logline)

Distributed Training with WORLD_SIZE: 1, RANK: 0, LOCAL_RANK: 0.
100%|███████���██| 26421880/26421880 [00:32<00:00, 819422.47it/s]
Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw

100%|██████████| 29515/29515 [00:00<00:00, 192512.06it/s]
Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

100%|██████████| 4422102/4422102 [00:01<00:00, 3157200.97it/s]
Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw

100%|██████████| 5148/5148 [00:00<00:00, 3510939.35it/s]
Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.312296
Train Epoch: 0 [1000/60000 (2%)]	Loss: 2.092805
Train Epoch: 0 [2000/60000 (3%)]	Loss: 1.597475
Train Epoch: 0 [3000/60000 (5%)]	Loss: 1.288376
Train Epoch: 0 [4000/60000 (7%)]	Loss: 0.876789
Train Epoch: 0 [5000/60000 (8%)]	Loss: 0.726644
Train Epoch: 0 [6000/60000 (10%)]	Loss: 0.593700

In [34]:
for j in TrainerClient().list_jobs():
    name = j.name 
    TrainerClient().delete_job(name)